### Black Scholes Option Pricing

In [30]:
%%file BS_Vanilla.m

function price = BS_Vanilla(C_P, S, X, r, T, vol, q)
    
    if nargin < 7, q=0;
    end
    
    d1 = (log(S/X)+(r-q+1/2*vol^2)*T)/(vol*sqrt(T));
    d2 = d1- vol*sqrt(T);
    
    if C_P == 'C'
        price =  S*exp(-q*T)*normcdf(d1)  - X*exp(-r*T)*normcdf(d2);
    elseif C_P == 'P'
        price = -S*exp(-q*T)*normcdf(-d1) + X*exp(-r*T)*normcdf(-d2);
    end

Created file '/Users/changsoo/Desktop/논문/Matlab/BS_Vanilla.m'.


In [33]:
Call = BS_Vanilla('C', 100, 100, 0.05, 0.5, 0.3, 0.01)


Call =

    9.3436



In [35]:
Put = BS_Vanilla('P', 100, 100, 0.05, 0.5, 0.3, 0.01)


Put =

    7.3733



In [37]:
[Call, Put] = blsprice(100, 100, 0.05, 0.5, 0.3, 0.01)


Call =

    9.3436


Put =

    7.3733



### Barone-Adesi and Whaley American Option Pricing Approximation

In [38]:
%%file nrcall.m
%Barone-Adesi and Whaley American Call Option

function result = nrcall(X, T, r, vol, q)

    %기초상수 계산
    b  = r - q;
    n  = 2 * b / vol^2;
    m  = 2 * r / vol^2;
    K  = 2 * r / (vol^2 * (1-exp(-r*T)));
    Q2 = (-(n-1) + sqrt((n-1)^2 + 4*K)) / 2;
    
    %초기 Si값 추정
    q2u = (-(n-1) + sqrt((n-1)^2 + 4*m))/2;
    Su  = X/(1-1/q2u);
    h2  = -(b*T + 2*vol*sqrt(T)) * X/(Su-X);
    Si  = X + (Su-X) * (1-exp(h2));
    
    %Newton-Raphson 방식으로 Si 계산
    d1 = (log(Si/X) + (b+vol^2/2)*T) / (vol*sqrt(T));
    LHS = Si-X;
    RHS = BS_Vanilla('C', Si, X, r, T, vol, q)  + (1-exp((b-r)*T)*normcdf(d1)) * Si/Q2;
    bi  = exp((b-r)*T) * normcdf(d1) * (1-1/Q2) + (1-exp((b-r)*T)*normpdf(d1) / (vol*sqrt(T)))/Q2;
    E   = 0.000001;
    
    %|LHS(Si) - RHS(Si)|/X의 값이 오차범위 안으로 나올 때까지 반복적 계산
    while abs(LHS-RHS) / X > E
        Si  = (X + RHS - bi * Si) / (1-bi);
        d1  = (log(Si/X) + (b+vol^2/2)*T) / (vol*sqrt(T));
        LHS = Si - X;
        RHS = BS_Vanilla('C', Si, X, r, T, vol, q)  + (1-exp((b-r)*T)*normcdf(d1)) * Si / Q2;
        bi  = exp((b-r)*T) * normcdf(d1) * (1-1/Q2) + (1-exp((b-r)*T)*normpdf(d1) / (vol*sqrt(T)))/Q2;
    end
    
    result = Si;


Created file '/Users/changsoo/Desktop/논문/Matlab/nrcall.m'.


In [42]:
%%file nrput.m
%Barone-Adesi and Whaley American Put Option

function result = nrput(X, T, r, vol, q)

    %기초상수 계산
    b  = r - q;
    n  = 2 * b / vol^2;
    m  = 2 * r / vol^2;
    K  = 2 * r / (vol^2 * (1-exp(-r*T)));
    Q1 = (-(n-1) + sqrt((n-1)^2 + 4*K)) / 2;
    
    %초기 Si값 추정
    q1u = (-(n-1) - sqrt((n-1)^2 + 4*m))/2;
    Su  = X/(1-1/q1u);
    h1  = (b*T - 2*vol*sqrt(T)) * X/(X-Su);
    Si  = Su + (X - Su) * exp(h1);
    
    %Newton-Raphson 방식으로 Si 계산
    d1 = (log(Si/X) + (b+vol^2/2)*T) / (vol*sqrt(T));
    LHS = X - Si;
    RHS = BS_Vanilla('P', Si, X, r, T, vol, q)    - (1-exp((b-r)*T)*normcdf(-d1)) * Si/Q1;
    bi  = -exp((b-r)*T) * normcdf(-d1) * (1-1/Q1) - (1+exp((b-r)*T)*normpdf(-d1) / (vol*sqrt(T)))/Q1;
    E   = 0.000001;
    
    %|LHS(Si) - RHS(Si)|/X의 값이 오차범위 안으로 나올 때까지 반복적 계산
    while abs(LHS-RHS) / X > E
        Si  = (X - RHS + bi * Si) / (1+bi);
        d1  = (log(Si/X) + (b+vol^2/2)*T) / (vol*sqrt(T));
        LHS = X - Si;
        RHS = BS_Vanilla('P', Si, X, r, T, vol, q)    - (1-exp((b-r)*T)*normcdf(-d1)) * Si / Q1;
        bi  = -exp((b-r)*T) * normcdf(-d1) * (1-1/Q1) - (1+exp((b-r)*T)*normpdf(-d1) / (vol*sqrt(T)))/Q1;
    end
    
    result = Si;


Created file '/Users/changsoo/Desktop/논문/Matlab/nrput.m'.


In [43]:
%%file BAW_Vanilla_Am.m
%Barone-Adesi and Whaley American Vanilla Option Pricing Approximation

function price = BAW_Vanilla_Am(C_P, S, X, r, T, vol, q)

    %배당수익률을 입력하지 않을 경우 0 처리
    if nargin < 7, q=0;
    end
    
    if C_P == 'C'
        if q == 0
            price = BS_Vanilla('C', S, X, r, T, vol, q);
        else
            %임계치 Si를 구하기 위해 nrcall 함수 사용
            Si = nrcall(X, T, r, vol, q);
            n  = 2 * (r-q) / vol^2;
            K  = 2 * r / (vol^2 * (1-exp(-r*T)));
            d1 = (log(Si/X) + (r-q+vol^2/2)*T) / (vol*sqrt(T));
            Q2 = (-(n-1) + sqrt((n-1)^2 + 4*K)) / 2;
            a2 = (Si/Q2) * (1-exp((-q) * T) * normcdf(d1));
            if S < Si
                price = BS_Vanilla('C', S, X, r, T, vol, q) + a2 * (S/Si)^Q2;
            else
                price = S - X;
            end
        end
    elseif C_P == 'P'
        Si = nrput(X, T, r, vol, q);
        n  = 2 * (r-q) / vol^2;
        K  = 2 * r / (vol^2 * (1-exp(-r*T)));
        d1 = (log(Si/X) + (r-q+vol^2/2) * T) / (vol*sqrt(T));
        Q1 = (-(n-1) - sqrt((n-1)^2 + 4*K)) / 2;
        a1 = -(Si / Q1) * (1-exp((-q) * T) * normcdf(-d1));
        if S > Si
            price = BS_Vanilla('P', S, X, r, T, vol, q) + a1 * (S/Si)^Q1;
        else
            price = X - S;
        end
    end

Created file '/Users/changsoo/Desktop/논문/Matlab/BAW_Vanilla_Am.m'.


In [46]:
put = BAW_Vanilla_Am('P', 100, 100, 0.05, 0.5, 0.3, 0.01)


put =

    7.5524



In [47]:
%%file BS_Barrier.m
%European Barrier Option Pricing(Closed Form)

function price = BS_Barrier(C_P, S, X, r, T, vol, B_type, B, q)

    %배당수익률을 입력하지 않을 경우 0 처리
    if nargin < 9, q=0;
    end
    
    %계산시 필요한 상수 계산
    if C_P == 'C'
        phi = 1;
    elseif C_P == 'P'
        phi = -1;
    end
    
    if B_type == 'di' | B_type == 'do'      %Down and in = di, Down and out = do
        eta = 1;
    elseif B_type == 'ui' | B_type == 'uo'  %Up and in = ui, Up and out = uo
        eta = -1;
    end
    
    mu = (r-q-vol^2/2)/vol^2;
    x1 = log(S/X) / (vol*sqrt(T)) + (1+mu)*vol*sqrt(T);
    x2 = log(S/B) / (vol*sqrt(T)) + (1+mu)*vol*sqrt(T);
    y1 = log(B^2 / (S*X)) / (vol*sqrt(T)) + (1+mu)*vol*sqrt(T);
    y2 = log(B/S) / (vol*sqrt(T)) + (1+mu)*vol*sqrt(T);
    
    %네 가지 기본식 계산
    f1 = phi*S*exp(-q*T)*normcdf(phi*x1) - phi*X*exp(-r*T)*normcdf(phi*x1 - phi*vol*sqrt(T));
    f2 = phi*S*exp(-q*T)*normcdf(phi*x2) - phi*X*exp(-r*T)*normcdf(phi*x2 - phi*vol*sqrt(T));
    f3 = phi*S*exp(-q*T)*(B/S)^(2*(mu+1))*normcdf(eta*y1) - phi*X*exp(-r*T)*(B/S)^(2*mu)*normcdf(eta*y1-eta*vol*sqrt(T));
    f4 = phi*S*exp(-q*T)*(B/S)^(2*(mu+1))*normcdf(eta*y2) - phi*X*exp(-r*T)*(B/S)^(2*mu)*normcdf(eta*y2-eta*vol*sqrt(T));
    
    %옵션 유형별 기본식 조합으로 옵션가치 평가
    if C_P == 'C'
        switch B_type
            case 'di'
                if X > B
                    price = f3;
                elseif X < B
                    price = f1 - f2 + f4;
                end
            case 'do'
                if X > B
                    price = f1 - f3;
                elseif X < B
                    price = f2 - f4;
                end
            case 'ui'
                if X > B
                    price = f1;
                elseif X < B
                    price = f2 - f3 + f4;
                end
            case 'uo'
                if X > B
                    price = 0;
                elseif X < B
                    price = f1 - f2 + f3 - f4;
                end
        end
    elseif C_P == 'P'
        switch B_type
            case 'di'
                if X > B
                    price = f2 - f3 + f4;
                elseif X < B
                    price = f1;
                end
            case 'do'
                if X > B
                    price = f1 - f2 + f3 - f4;
                elseif X < B
                    price = 0;
                end
            case 'ui'
                if X > B
                    price = f1 - f2 + f4;
                elseif X < B
                    price = f3;
                end
            case 'uo'
                if X > B
                    price = f2 - f4;
                elseif X < B
                    price = f1 - f3;
                end
        end
    end
    
    if price <=0 
        price=0;
    end

Created file '/Users/changsoo/Desktop/논문/Matlab/BS_Barrier.m'.


In [50]:
%Knock-Out 콜옵션가격
a = BS_Barrier('C', 100, 100, 0.05, 0.5, 0.3, 'uo', 130, 0.01)


a =

    2.8481



In [51]:
%Knock-In 콜옵션 가격
b = BS_Barrier('C', 100, 100, 0.05, 0.5, 0.3, 'ui', 130, 0.01)


b =

    6.4955



In [52]:
c = BS_Vanilla('C', 100, 100, 0.05, 0.5, 0.3, 0.01)


c =

    9.3436



In [53]:
%바닐라 옵션가격 = Knock-Out 콜옵션가격 + Knock-In 콜옵션 가격
check = a+b


check =

    9.3436



In [67]:
%Knock-out 풋옵션 가격
a = BS_Barrier('P', 100, 100, 0.05, 0.5, 0.3, 'do', 70, 0.01)


a =

    4.6447



In [68]:
%Knock-In 풋옵션 가격
b = BS_Barrier('P', 100, 100, 0.05, 0.5, 0.3, 'di', 70, 0.01)


b =

    2.7287



In [69]:
p = BS_Vanilla('P', 100, 100, 0.05, 0.5, 0.3, 0.01)


p =

    7.3733



In [71]:
%바닐라 풋옵션가격 = Knock-Out 풋옵션 가격 + Knock-In 풋옵션 가격
check = a+b


check =

    7.3733

